In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path = '/gdrive/My Drive/Online_MLW : Sept 2020/Data/'

In [ ]:
data=pd.read_csv(path+'paint_mnist.csv',header=None)

In [ ]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
data.shape

(3599, 785)

In [ ]:
cols=data.shape[1]
print("columns = {}".format(cols))

columns = 785


In [ ]:
y=data.iloc[:,:1]
x=data.iloc[:,1:cols]

In [ ]:
x.shape, y.shape

((3599, 784), (3599, 1))

In [ ]:
b = np.unique(y)  #finding all unique classes in the output
print(b)

[0 1 2 3]


In [ ]:
classes = b.size
print(classes)

4


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=2)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2699, 784), (900, 784), (2699, 1), (900, 1))

In [ ]:
x=np.array(x_train)
y=np.array(y_train)

In [ ]:
rows = x.shape[0] #Finding the number of Rows (data samples)
x = np.insert(x, 0, values=np.ones(rows), axis=1) #inserting ones into x

x.shape

(2699, 785)

In [ ]:
params = x.shape[1]               #Finding the number of features
all_theta = np.zeros((classes, params)) # here 4 denotes number of labels (0 to 3)

all_theta.shape 

(4, 785)

In [ ]:
def sigmoid(z):    #returns values between zero to one
    return 1 / (1 + np.exp(-z))

In [ ]:
sigmoid(10000000000000000)

1.0

In [ ]:
def computeCost(theta, x, y):
    theta = np.matrix(theta)
    x = np.matrix(x)
    y = np.matrix(y)
    
    #(-y*log(g(z))) --> when expected output (y)=1
    first = np.multiply(-y, np.log(sigmoid(x * theta.T))) 
    
    #((1-y)*(1-log(g(z)))) --> when expected output (y)=0
    second = np.multiply((1 - y), np.log(1 - sigmoid(x * theta.T)))
    
    cost = np.sum(first - second)
    cost = cost/(2*len(x)) 
                 
    return cost

In [ ]:
cost= computeCost(all_theta, x, y)
print("cost = {}".format(cost))

cost = 1.3862943611198906


In [ ]:
def gradient(theta, x, y):
    
    theta = np.matrix(theta)
    x = np.matrix(x)
    y = np.matrix(y)
    learning_rate = 1
        
    error = sigmoid(x * theta.T) - y
    grad = learning_rate*((x.T * error) / len(x))
    
    return grad

In [ ]:
from scipy.optimize import minimize
def one_vs_all(x, y, classes, new_theta):
    params = x.shape[1]
    rows = x.shape[0]
      
    for i in range(0, classes):
        theta = new_theta[i,:]
        
        y_i = np.array([1 if label == i else 0 for label in y]) 
        y_i = np.reshape(y_i, (rows, 1))
                
        # minimize the objective function  --> Gradient descent
        fmin = minimize(fun=computeCost, x0=theta, args=(x, y_i), method='TNC', jac=gradient)
        all_theta[i,:] = fmin.x
        
        print(fmin.fun)
    
    return all_theta 

In [ ]:
new_theta = one_vs_all(x,y, classes, all_theta)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in multiply
  # Remove the CWD from sys.path while we load stuff.


0.012821478058623069
0.06081297680962059
0.029572792746861365
0.03428951604782847


In [ ]:
print('new_theta = {}'.format(new_theta))

new_theta = [[-0.00277529  0.          0.         ... -0.00062769 -0.00053347
  -0.00053347]
 [ 0.00291599  0.          0.         ... -0.00083366  0.00032797
   0.00032797]
 [-0.01599503  0.          0.         ...  0.0124741   0.00179744
   0.00179744]
 [-0.0116006   0.          0.         ... -0.01220392 -0.00930566
  -0.00930566]]


In [ ]:
def predict_all(x, theta): 
   
    # convert to matrices
    x = np.matrix(x)
    theta = np.matrix(theta)
    
    # run the model on each training instance
    h = sigmoid(x * theta.T)
    
    # return the column index that has the maximum confidence
    h_argmax = np.argmax(h, axis=1)
    
    return h_argmax, h

In [ ]:
y_pred, h = predict_all(x, new_theta)
#print(y_pred)

In [ ]:
def acc(y_pred,y):
  correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
  accuracy = (sum(correct)) / (len(correct))
  return accuracy

In [ ]:
accuracy = acc(y_pred,y)

print ("Training data accuracy={} %".format(accuracy * 100))

Training data accuracy=97.44349759170063 %


In [ ]:
t_x = np.array(x_test)
t_y = np.array(y_test)

In [ ]:
t_rows = t_x.shape[0] #Finding the number of Rows (data samples)
t_x = np.insert(t_x, 0, values=np.ones(t_rows), axis=1) #inserting ones into x

In [ ]:
t_y_pred, h = predict_all(t_x, new_theta)

In [ ]:
print ("Test data accuracy={} %".format(acc(t_y_pred, t_y) * 100))

Test data accuracy=76.44444444444444 %


In [ ]:
from PIL import Image
from PIL import ImageOps
from IPython.display import display

Im= Image.open(path+'pic.png')
display(Im)

In [ ]:
Im=Im.convert('L')    # convert to grayScale
Im = ImageOps.invert(Im) #invert the Image
display(Im)

In [ ]:
Im = np.array(Im)
Im = np.ravel(Im)
Im.shape

(784,)

In [ ]:
Im= Im.reshape(1,-1)

input_x = np.insert(Im, 0, 1, axis=1)

In [ ]:
input_x.shape

(1, 785)

In [ ]:
pred, h = predict_all(input_x, new_theta)


print ("confidence matrix={} ".format(h))

print('predicted class is : {}'.format(pred.item()))

print ("confidence={} %".format(((h[0,pred]) * 100).item()))

confidence matrix=[[3.93024239e-05 9.91454170e-01 1.19632150e-03 1.13115907e-06]] 
predicted class is : 1
confidence=99.1454169586249 %
